We will import the spacetime variables module.  This module contains all of the indexing and symbolic manipulation tools we will use to write our equations and convert them to AMReX executable code.

In [1]:
from SpacetimeVar import *
from sympy import Eijk, LeviCivita

In [2]:
DiffOrder = 4  #We'll use 4th order finite differencing for the equations of motion.
nghostcells = 4  #Number of ghost cells needed for finite differencing on the boundaries.
KOsigma = 0.1   #For Kreiss Oliger dissipation terms.

There are seven source header files we need to generate.

"ET_Integration_Rhs_K.H" contains the right hand sides of our equations of motion.
"ET_Integration_Init_K.H" will initialize our evolution variables to some specified analytic expression. 
"ET_Integration_Variables.H" contains the set of evolution variable names.

"ET_Integration_Diag_K.H" will contain the definitons of any diagnostic variables we wish to keep track of. Things like the Hamiltonian constraint, and the Momentum constraints.  For this case we will store the definitions of the evolution variables for the Z4c solver.  We will specify how the lapse, chi, curvatures, etc. are written in terms of the variable u.

"ET_Integration_Diagnostic_Variables_K.H" contains the set of diagnostic variable names.
"ET_Integration_Post_Update_K.H" is an optional file that will specify any modifications to the variables made after each timestep.  Here it is unused, but will be needed for the Z4c code generator to enforce constraints on the conformal factor, and the tracelessness of Atilde.

"ET_Integration_Setup.H" specifies the namespaces for the variable and diagnostic names.  It also specifies the number of ghost cells needed.

In [3]:
dim = 3  #Working in 3 spatial dimensions
path = "../Source/InitialDataSolver/"  #This is where we will want or generated source code to go.

We begin by defining the RHS for the relaxation procedure for our variable u.

In [4]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")  #Open the ET_Integration_Rhs_K.H file.

In [5]:
#Headers used in RHS file.

RHS_Header = """#ifndef ET_INTEGRATION_RHS_K_H 
#define ET_INTEGRATION_RHS_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_rhs(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& rhs_fab, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, 
        const amrex::GeometryData& geom) noexcept 
{
        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(2) + domain_xlo[2]; 
"""

fileRHS.write(RHS_Header)

705

We specify the variables we intend to evolve.  In this case we have only on variable u.

In [6]:
stVar.decl = [] #We will hold a list of evolution variable names that we cycle through when converting sympy expressions to C code.
u = stVar('u', declare = True)  #In this case we only have one variable to evolve, u.

In [7]:
fileRHS.write(u.AMReXDeclare()) #This line will declare u symbolically from the mesh data in ET_Integration_Rhs_K.H.

52

It is convenient at this point to produce the "ET_Integration_Variables.H" file from the list of variable names in stVar.decl.  In this case only a single evolution variable name u will be stored.

In [8]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write("names = {")
for itr in range(len(stVar.decl)-1):
    fileVARS.write("\""+stVar.decl[itr]+"\", ")
fileVARS.write("\""+stVar.decl[len(stVar.decl)-1]+"\"")
fileVARS.write("};")
fileVARS.close()

Continuing with the RHS, the following lines will define ammrex::Real variables for the derivatives of u.  DstVar is a function that takes as arguments the stVar variable, the derivative type "difftype" (i.e. first or second derivative). CnUpDnRank1 specifies whether we use upwinded 'up', downwinded 'dn', or (default) central 'cn' finite differencing.  'orderD' specifies the order of finite differencing.

In [9]:
fileRHS.write(DstVar(u, difftype = 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(u, difftype = 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(u, 'KO', orderD = DiffOrder).AMReXDeclare())

835

We will now define a few intermediate variables that are needed for the evolution of u.  We wish to solve the initial data of two blackholes that are in orbit.  To do this we will need to specify their bare masses M1 and M2, their initial puncture positions c1 and c2, and their initial momenta P1 and P2. For fun we can also give them some spin S1 and S2.

In [10]:
x = stVarRank1('x')  #Defining symbols for the grid point positions.  They are already declared in the header.
r = stVar('r')  #Define symbol for radius from the center of our coordinate system.
r.var = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)  #Defining how r is related to x0, x1, and x2.

#We place two black holes at postions along the x0 = x axis.
c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.var = np.array([1.168642873,0,0])
c2.var = np.array([-1.168642873,0,0])

#Bare masses
M1 = stVar('M1')
M2 = stVar('M2')

M1.var = 0.453*0.66*2
M2.var = 0.453*0.33*2

#Momenta
P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.var = np.array([0,0.2331917498,0])
P2.var = np.array([0,-0.2331917498,0])

S1 = stVarRank1('S1')
S2 = stVarRank1('S2')

S1.var = [0,0,0]
S2.var = [0,0,0]

#To simplify expressions below we can specify intermediate coordinates defined relative to the black hole centers.
xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.var[i] = x.symb[i]-c1.symb[i]
    xc2.var[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.var = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.var = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.var[i] = xc1.symb[i]/rc1.symb
    n2.var[i] = xc2.symb[i]/rc2.symb

We now declare the expressions in the RHS file as 'amrex::Real's.

In [11]:
fileRHS.write(r.AMReXReal())
fileRHS.write(c1.AMReXReal())
fileRHS.write(c2.AMReXReal())
fileRHS.write(M1.AMReXReal())
fileRHS.write(M2.AMReXReal())
fileRHS.write(P1.AMReXReal())
fileRHS.write(P2.AMReXReal())
fileRHS.write(S1.AMReXReal())
fileRHS.write(S2.AMReXReal())
fileRHS.write(xc1.AMReXReal())
fileRHS.write(xc2.AMReXReal())
fileRHS.write(rc1.AMReXReal())
fileRHS.write(rc2.AMReXReal())
fileRHS.write(n1.AMReXReal())
fileRHS.write(n2.AMReXReal())

108

Let us now define the curvature tensors Abar by their puncture expressions.  See Baumgarte's book Chapter 12 for definitions.

In [12]:
Abar1UU = stVarRank2('Abar1UU')
Abar2UU = stVarRank2('Abar2UU')
for i in range(3):
    for j in range(3):
        Abar1UU.var[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Abar2UU.var[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Abar1UU.var[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Abar2UU.var[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                Abar1UU.var[i][j] +=3/(rc1.symb**3)*(n1.symb[i]*Eijk(j,k,l)*S1.symb[k]*n1.symb[l]+n1.symb[j]*Eijk(i,k,l)*S1.symb[k]*n1.symb[l])
                Abar2UU.var[i][j] +=3/(rc2.symb**3)*(n2.symb[i]*Eijk(j,k,l)*S2.symb[k]*n2.symb[l]+n2.symb[j]*Eijk(i,k,l)*S2.symb[k]*n2.symb[l])

In [13]:
AbarUU = stVarRank2('AbarUU')
for i in range(3):
    for j in range(3):
        AbarUU.var[i][j] += Abar1UU.symb[i][j]+Abar2UU.symb[i][j]

In [14]:
fileRHS.write(Abar1UU.AMReXReal())
fileRHS.write(Abar2UU.AMReXReal())
fileRHS.write(AbarUU.AMReXReal())

486

alpha and beta will contain their corresponding analytic expressions in terms of Abar and the masses.

In [15]:
alphafunc = stVar('alphafunc')

alphafunc.var = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [16]:
fileRHS.write(alphafunc.AMReXReal())

75

In [17]:
beta = stVar('beta')

for i in range(3):
    for j in range(3):
        beta.var += 1/8*(alphafunc.symb**7)*AbarUU.symb[i][j]*AbarUU.symb[i][j]

In [18]:
fileRHS.write(beta.AMReXReal())

507

We are now ready to write the equation of motion for u in terms of Abar, alpha and beta defined above.

In [19]:
RHS_u = stVar('u')

RHS_u.var = 0
for i in range(3):
    RHS_u.var += Dsymb(u.symb,str(i)+str(i))
    
RHS_u.var += beta.symb*(alphafunc.symb*(1+u.symb)+1)**(-7)

RHS_u.var += Dsymb(u.symb,'KO')

In [20]:
fileRHS.write(RHS_u.AMReXRHS())

107

We can now finish and close ET_Integration_Rhs_K.H.

In [21]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

Let us now consider the output expressions for the Z4c variables that we will want to import into the Z4c solver, using the solutions for u.  We will contain these definitions in the diagnostic files.

In [22]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")

In [23]:
DIAG_Header = """#ifndef ET_INTEGRATION_DIAG_K_H 
#define ET_INTEGRATION_DIAG_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_diagnostics(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& diag, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx,
        const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(2) + domain_xlo[2];
"""

fileDIAG.write(DIAG_Header)

711

In [24]:
fileDIAG.write(u.AMReXDeclare())

52

We proceed similarily to the RHS expressions above.

In [25]:
x = stVarRank1('x')
r = stVar('r')
r.var = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)

c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.var = np.array([1.168642873,0,0])
c2.var = np.array([-1.168642873,0,0])

M1 = stVar('M1')
M2 = stVar('M2')

M1.var = 0.453*0.66*2
M2.var = 0.453*0.33*2

P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

S1 = stVarRank1('S1')
S2 = stVarRank1('S2')

S1.var = [0,0,0]
S2.var = [0,0,0]

P1.var = np.array([0,0.2331917498,0])
P2.var = np.array([0,-0.2331917498,0])

xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.var[i] = x.symb[i]-c1.symb[i]
    xc2.var[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.var = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.var = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

eps1 = stVar('eps1')
eps2 = stVar('eps2')

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.var[i] = xc1.symb[i]/rc1.symb
    n2.var[i] = xc2.symb[i]/rc2.symb

In [26]:
fileDIAG.write(r.AMReXReal())
fileDIAG.write(c1.AMReXReal())
fileDIAG.write(c2.AMReXReal())
fileDIAG.write(M1.AMReXReal())
fileDIAG.write(M2.AMReXReal())
fileDIAG.write(P1.AMReXReal())
fileDIAG.write(P2.AMReXReal())
fileDIAG.write(S1.AMReXReal())
fileDIAG.write(S2.AMReXReal())
fileDIAG.write(xc1.AMReXReal())
fileDIAG.write(xc2.AMReXReal())
fileDIAG.write(rc1.AMReXReal())
fileDIAG.write(rc2.AMReXReal())
fileDIAG.write(n1.AMReXReal())
fileDIAG.write(n2.AMReXReal())

108

In [27]:
Abar1UU = stVarRank2('Abar1UU')
Abar2UU = stVarRank2('Abar2UU')
for i in range(3):
    for j in range(3):
        Abar1UU.var[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Abar2UU.var[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Abar1UU.var[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Abar2UU.var[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                Abar1UU.var[i][j] +=3/(rc1.symb**3)*(n1.symb[i]*Eijk(j,k,l)*S1.symb[k]*n1.symb[l]+n1.symb[j]*Eijk(i,k,l)*S1.symb[k]*n1.symb[l])
                Abar2UU.var[i][j] +=3/(rc2.symb**3)*(n2.symb[i]*Eijk(j,k,l)*S2.symb[k]*n2.symb[l]+n2.symb[j]*Eijk(i,k,l)*S2.symb[k]*n2.symb[l])

In [28]:
AbarUU = stVarRank2('AbarUU')
for i in range(3):
    for j in range(3):
        AbarUU.var[i][j] += Abar1UU.symb[i][j]+Abar2UU.symb[i][j]

In [29]:
fileDIAG.write(Abar1UU.AMReXReal())
fileDIAG.write(Abar2UU.AMReXReal())
fileDIAG.write(AbarUU.AMReXReal())

486

In [30]:
alphafunc = stVar('alphafunc')

alphafunc.var = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [31]:
fileDIAG.write(alphafunc.AMReXReal())

75

Here it will be useful to define and declare 'psi' from which the Z4c variables will be initialize from.

In [32]:
psi = stVar('psi')

psi.var = 1+alphafunc.symb**(-1)+u.symb

In [33]:
fileDIAG.write(psi.AMReXReal())

49

In [34]:
stVar.declDiag = []  #We want a list of declared variables that we will cycle through when we convert our RHS equations.

chi = stVar('chi',declareDiag = True) #Scalar variables like chi can be declared with stVar.
gamtildeLL = stVarRank2('gamtildeLL', sym = 'sym01', declareDiag = True)  #gamtildeLL is rank 2, so it is declared with stVarRank2
Khat = stVar('Khat',declareDiag = True)
AtildeLL = stVarRank2('AtildeLL', sym = 'sym01',declareDiag = True)
theta = stVar('theta',declareDiag = True)
GamtildeU = stVarRank1('GamtildeU',declareDiag = True)

alpha = stVar('alpha',declareDiag = True)
betaU = stVarRank1('betaU',declareDiag = True)

In [35]:
fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write("names = {")
for itr in range(len(stVar.declDiag)-1):
    fileDIAGVARS.write("\""+stVar.declDiag[itr]+"\", ")
fileDIAGVARS.write("\""+stVar.declDiag[len(stVar.declDiag)-1]+"\"")
fileDIAGVARS.write("};")
fileDIAGVARS.close()

We are now ready to write out the initial expressions for the Z4c variables using the expressions for psi above.

In [36]:
chi.var = psi.symb**(-4)

In [37]:
fileDIAG.write(chi.AMReXDiag())

55

In [38]:
gamtildeLL.var = [[1,0,0],[0,1,0],[0,0,1]]

In [39]:
fileDIAG.write(gamtildeLL.AMReXDiag())

288

In [40]:
Khat.var = 0

In [41]:
fileDIAG.write(Khat.AMReXDiag())

40

In [42]:
AtildeLL.var = (psi.symb**(-6))*AbarUU.symb

In [43]:
fileDIAG.write(AtildeLL.AMReXDiag())

420

In [44]:
theta.var = 0

In [45]:
fileDIAG.write(theta.AMReXDiag())

41

In [46]:
GamtildeU.var = [0,0,0]

In [47]:
fileDIAG.write(GamtildeU.AMReXDiag())

138

In [48]:
alpha.var = psi.symb**(-2)

In [49]:
fileDIAG.write(alpha.AMReXDiag())

57

In [50]:
betaU.var = [0,0,0]

In [51]:
fileDIAG.write(betaU.AMReXDiag())

126

In [52]:
fileDIAG.write("}\n")
fileDIAG.write("#endif")
fileDIAG.close()

In [53]:
fileSETUP = open(path+"ET_Integration_Setup.H", "w+")

In [54]:
fileSETUP.write("#ifndef ET_INTEGRATION_SETUP_K_H \n")
fileSETUP.write("#define ET_INTEGRATION_SETUP_K_H \n\n")

fileSETUP.write("#include <AMReX_REAL.H> \n")
fileSETUP.write("#include <AMReX_Array4.H> \n\n")
    
fileSETUP.write("namespace Idx { \n")
fileSETUP.write("         enum ETIndexes {")
    
Idx_string = ""
for itr in stVar.decl:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("namespace Diag { \n")
fileSETUP.write("         enum DiagnosticIndexes {")
    
Idx_string = ""
for itr in stVar.declDiag:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("#define NUM_GHOST_CELLS "+str(nghostcells)+"\n\n")
fileSETUP.write("#endif")

fileSETUP.close()

Since there is no post timestep updating for this case we will leave the body of the PostUpdate file blank.

In [55]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")

In [56]:
PostUpdate_Header = """#ifndef ET_INTEGRATION_POST_UPDATE_K_H 
#define ET_INTEGRATION_POST_UPDATE_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_post_update(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        const amrex::GeometryData& geom) noexcept 
{
"""

filePOST.write(PostUpdate_Header)

348

In [57]:
filePOST.write("}\n")
filePOST.write("#endif")
filePOST.close()

We finally initialize u.  Here setting the initial value for u to zero works as a starting point in the relaxation procedure.

In [58]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")

In [59]:
Init_Header = """#ifndef ET_INTEGRATION_INIT_K_H 
#define ET_INTEGRATION_INIT_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_init(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        amrex::Real time, const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(2) + domain_xlo[2]; 
"""

fileINIT.write(Init_Header)

605

In [60]:
uInit = stVar('u')

In [61]:
uInit.var = 0

In [62]:
fileINIT.write(uInit.AMReXInit())

41

In [63]:
fileINIT.write("}\n")
fileINIT.write("#endif")
fileINIT.close()